# Lab 3 - Jan Banot

## Zadanie:
ybierz dowolny zbiór danych transakcyjnych z publicznego repozytorium (Kaggle lub UCI Machine Learning Repository). Upewnij się, że Twój zbiór nie pokrywa się z wybranymi przez pozostałych studentów w grupie.

Przeprowadź analizę reguł asocjacyjnych, obejmującą:
- wygenerowanie zbiorów częstych i reguł asocjacyjnych dla trzech różnych par progów wsparcia i ufności,
- porównanie wpływu zmiany progów na liczbę i jakość reguł,
- wskazanie najciekawszych reguł oraz ich potencjalnego znaczenia w kontekście danych.
Przygotuj sprawozdanie z wykonanych prac (skrypt Python z komentarzami, nagrany film lub raport PDF), zawierające opis wybranego zbioru, zastosowaną metodologię, wyniki analizy oraz wnioski.

## Wybrany zbiór danych
https://www.kaggle.com/datasets/aliessamali/ecommerce
Zbiór danych został skrócony do 10tys. elementów w celach przyspieszenia obliczeń.

In [ ]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# ================================
# 1. WCZYTANIE I EKSPLORACJA DANYCH
# ================================


def load_and_explore_data(file_path):
    """
    Wczytuje i eksploruje dane transakcyjne
    """
    print("=== WCZYTYWANIE I EKSPLORACJA DANYCH ===")

    # Wczytanie danych
    df = pd.read_csv(file_path)

    print(f"Kształt danych: {df.shape}")
    print(f"Liczba unikalnych faktur: {df['InvoiceNo'].nunique()}")
    print(f"Liczba unikalnych produktów: {df['StockCode'].nunique()}")
    print(f"Liczba unikalnych klientów: {df['CustomerID'].nunique()}")

    # Sprawdzenie brakujących danych
    print("\nBrakujące dane:")
    print(df.isnull().sum())

    return df

In [ ]:
# ================================
# 2. PRZYGOTOWANIE DANYCH
# ================================


def prepare_transaction_data(df):
    """
    Przygotowuje dane do analizy reguł asocjacyjnych
    """
    print("\n=== PRZYGOTOWANIE DANYCH ===")

    # Usunięcie wierszy z brakującymi danymi kluczowymi
    df_clean = df.dropna(subset=["InvoiceNo", "StockCode", "Description"])

    # Usunięcie transakcji anulowanych (zaczynających się od 'C')
    df_clean = df_clean[~df_clean["InvoiceNo"].astype(str).str.startswith("C")]

    # Usunięcie produktów o ujemnej ilości
    df_clean = df_clean[df_clean["Quantity"] > 0]

    print(f"Dane po czyszczeniu: {df_clean.shape}")

    # Grupowanie po fakturze i produktach - utworzenie macierzy transakcyjnej
    basket = (
        df_clean.groupby(["InvoiceNo", "Description"])["Quantity"]
        .sum()
        .unstack()
        .fillna(0)
    )

    # Konwersja na format binarny (0/1)
    basket_sets = basket.applymap(lambda x: 1 if x > 0 else 0)

    print(f"Macierz transakcyjna: {basket_sets.shape}")

    return basket_sets, df_clean

In [ ]:
# ================================
# 3. ANALIZA REGUŁ ASOCJACYJNYCH
# ================================


def analyze_association_rules(basket_sets, min_support_values, min_confidence_values):
    """
    Przeprowadza analizę reguł asocjacyjnych dla różnych progów
    """
    results = {}

    for i, (min_sup, min_conf) in enumerate(
        zip(min_support_values, min_confidence_values)
    ):
        print(f"\n=== ANALIZA {i + 1}: Support={min_sup}, Confidence={min_conf} ===")

        # Znajdowanie częstych zbiorów elementów
        frequent_itemsets = apriori(basket_sets, min_support=min_sup, use_colnames=True)

        if len(frequent_itemsets) == 0:
            print(f"Brak częstych zbiorów dla support={min_sup}")
            results[f"config_{i + 1}"] = {
                "support": min_sup,
                "confidence": min_conf,
                "frequent_itemsets": pd.DataFrame(),
                "rules": pd.DataFrame(),
                "stats": {"frequent_count": 0, "rules_count": 0},
            }
            continue

        print(f"Liczba częstych zbiorów: {len(frequent_itemsets)}")

        # Generowanie reguł asocjacyjnych
        rules = association_rules(
            frequent_itemsets, metric="confidence", min_threshold=min_conf
        )

        print(f"Liczba reguł: {len(rules)}")

        # Dodanie dodatkowych metryk
        if len(rules) > 0:
            rules = rules.round(4)
            rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
            rules["consequent_len"] = rules["consequents"].apply(lambda x: len(x))

        # Zapisanie wyników
        results[f"config_{i + 1}"] = {
            "support": min_sup,
            "confidence": min_conf,
            "frequent_itemsets": frequent_itemsets,
            "rules": rules,
            "stats": {
                "frequent_count": len(frequent_itemsets),
                "rules_count": len(rules),
            },
        }

        # Wyświetlenie statystyk
        if len(rules) > 0:
            print(f"Średnie wsparcie: {rules['support'].mean():.4f}")
            print(f"Średnia ufność: {rules['confidence'].mean():.4f}")
            print(f"Średni lift: {rules['lift'].mean():.4f}")

    return results

In [ ]:
# ================================
# 4. PORÓWNANIE WYNIKÓW
# ================================


def compare_results(results):
    """
    Porównuje wyniki dla różnych progów
    """
    print("\n=== PORÓWNANIE WYNIKÓW ===")

    comparison_data = []
    for config, data in results.items():
        comparison_data.append(
            {
                "Konfiguracja": config,
                "Support": data["support"],
                "Confidence": data["confidence"],
                "Liczba częstych zbiorów": data["stats"]["frequent_count"],
                "Liczba reguł": data["stats"]["rules_count"],
            }
        )

    comparison_df = pd.DataFrame(comparison_data)
    print(comparison_df.to_string(index=False))

    return comparison_df

In [ ]:
# ================================
# 5. WIZUALIZACJA
# ================================


def create_visualizations(results, comparison_df):
    """
    Tworzy wizualizacje wyników
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

    # 1. Porównanie liczby reguł
    axes[0, 0].bar(comparison_df["Konfiguracja"], comparison_df["Liczba reguł"])
    axes[0, 0].set_title("Liczba reguł dla różnych progów")
    axes[0, 0].set_xlabel("Konfiguracja")
    axes[0, 0].set_ylabel("Liczba reguł")

    # 2. Porównanie liczby częstych zbiorów
    axes[0, 1].bar(
        comparison_df["Konfiguracja"], comparison_df["Liczba częstych zbiorów"]
    )
    axes[0, 1].set_title("Liczba częstych zbiorów dla różnych progów")
    axes[0, 1].set_xlabel("Konfiguracja")
    axes[0, 1].set_ylabel("Liczba częstych zbiorów")

    # 3. Rozkład lift dla pierwszej konfiguracji z regułami
    config_with_rules = None
    for config, data in results.items():
        if len(data["rules"]) > 0:
            config_with_rules = config
            break

    if config_with_rules:
        rules = results[config_with_rules]["rules"]
        axes[1, 0].hist(rules["lift"], bins=20, alpha=0.7)
        axes[1, 0].set_title(f"Rozkład wartości Lift ({config_with_rules})")
        axes[1, 0].set_xlabel("Lift")
        axes[1, 0].set_ylabel("Częstość")

    # 4. Scatter plot support vs confidence
    if config_with_rules:
        scatter = axes[1, 1].scatter(
            rules["support"],
            rules["confidence"],
            c=rules["lift"],
            cmap="viridis",
            alpha=0.7,
        )
        axes[1, 1].set_title(f"Support vs Confidence ({config_with_rules})")
        axes[1, 1].set_xlabel("Support")
        axes[1, 1].set_ylabel("Confidence")
        plt.colorbar(scatter, ax=axes[1, 1], label="Lift")

    plt.tight_layout()
    plt.show()

In [ ]:
# ================================
# 6. ANALIZA NAJCIEKAWSZYCH REGUŁ
# ================================


def analyze_top_rules(results):
    """
    Analizuje najciekawsze reguły
    """
    print("\n=== ANALIZA NAJCIEKAWSZYCH REGUŁ ===")

    for config, data in results.items():
        rules = data["rules"]
        if len(rules) == 0:
            continue

        print(
            f"\n--- Konfiguracja: {config} (Support={data['support']}, Confidence={data['confidence']}) ---"
        )

        # Top 5 reguł według lift
        print("TOP 5 REGUŁ WEDŁUG LIFT:")
        top_lift = rules.nlargest(5, "lift")[
            ["antecedents", "consequents", "support", "confidence", "lift"]
        ]
        for idx, row in top_lift.iterrows():
            antecedent = ", ".join(list(row["antecedents"]))
            consequent = ", ".join(list(row["consequents"]))
            print(f"  {antecedent} → {consequent}")
            print(
                f"    Support: {row['support']:.4f}, Confidence: {row['confidence']:.4f}, Lift: {row['lift']:.4f}\n"
            )

        # Top 5 reguł według confidence
        print("TOP 5 REGUŁ WEDŁUG CONFIDENCE:")
        top_conf = rules.nlargest(5, "confidence")[
            ["antecedents", "consequents", "support", "confidence", "lift"]
        ]
        for idx, row in top_conf.iterrows():
            antecedent = ", ".join(list(row["antecedents"]))
            consequent = ", ".join(list(row["consequents"]))
            print(f"  {antecedent} → {consequent}")
            print(
                f"    Support: {row['support']:.4f}, Confidence: {row['confidence']:.4f}, Lift: {row['lift']:.4f}\n"
            )


In [ ]:
# ================================
# 7. INTERPRETACJA BIZNESOWA
# ================================


def business_interpretation(results, df_clean):
    """
    Interpretacja wyników w kontekście biznesowym
    """
    print("\n=== INTERPRETACJA BIZNESOWA ===")

    print("ZNACZENIE METRYK:")
    print("- Support: Jak często dany zbiór produktów występuje w transakcjach")
    print("- Confidence: Prawdopodobieństwo zakupu produktu B przy zakupie produktu A")
    print("- Lift: Czy produkty są kupowane razem częściej niż przypadkowo")
    print("  * Lift > 1: Produkty kupowane razem częściej niż przypadkowo")
    print("  * Lift = 1: Brak zależności")
    print("  * Lift < 1: Produkty kupowane razem rzadziej niż przypadkowo")

    print("\nZALECANIA:")
    print("1. Reguły z wysokim lift (>1.5) wskazują na silne powiązania produktów")
    print("2. Wysokie confidence wskazuje na niezawodne predykcje zakupów")
    print("3. Odpowiedni support zapewnia statystyczną istotność reguł")

    # Analiza najpopularniejszych produktów
    print("\nNAJPOPULARNIEJSZE PRODUKTY:")
    top_products = df_clean["Description"].value_counts().head(10)
    print(top_products)

In [ ]:
file_path = "/Users/janbanot/Dev/uni/msc-cs-code/sem2/ADWB/data/e-commerce-dataset.csv"

# Wczytanie i eksploracja danych
df = load_and_explore_data(file_path)

# Przygotowanie danych
basket_sets, df_clean = prepare_transaction_data(df)

# Definicja trzech par progów (support, confidence)
min_support_values = [0.4, 0.3, 0.2, 0.1, 0.05]
min_confidence_values = [0.6, 0.5, 0.3, 0.2, 0.1]

# Analiza reguł asocjacyjnych
results = analyze_association_rules(basket_sets, min_support_values, min_confidence_values)

# Porównanie wyników
comparison_df = compare_results(results)

# Wizualizacja
create_visualizations(results, comparison_df)

# Analiza najciekawszych reguł
analyze_top_rules(results)

# Interpretacja biznesowa
business_interpretation(results, df_clean)
